In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

from bs4 import BeautifulSoup
import requests

from datetime import timedelta, date

import itertools

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
dates=[] #pasirinktų datų sąrašas nuskaitomiems duomenims

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2018, 1, 1) 
end_dt = date(2018, 2, 27)


for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))

dates=[d for d in dates if d not in ['2018-02-19', '2018-03-02']]
#Policijos departamento informacija. 
#AttributeError: 'NoneType' object has no attribute 'text'

In [3]:
dfs2 = [] #Lentelių sąrašas paros duomenų lentelei apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus sukurti
#dfs2 = [] Lentelių sąrašas paros gaisrų aprašymų lentelei sukurti

for dateurl in dates:
    url = 'http://www.vpgt.lt/vpgt/m/m_events_summary/wfiles/e_'+dateurl+'.htm'
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c,'lxml')
    table = soup.find('table')
    rows = table.find_all('tr')
    
    data0=[] #duomenų sąrašas
                 
            
    for tr in rows:
        cols = tr.find_all('td')
        for td in cols:
            if td.attrs["style"] == "border:none":
                td.decompose()
            else:
                text1 = ""
                for text0 in td.find_all("p"):
                    text1 += text0.text + " "
                data0.append(text1.rstrip())

                
    #Paros duomenys apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus
    data0 = [w.replace('\r\n ', '') for w in data0]
    data0 = [w.replace('\n', ' ') for w in data0]
    
    #Paros gaisrų aprašymų duomenys
    data2 = data0[data0.index('2. Gaisrų aprašymas')+1:data0.index('3. Gelbėjimo darbų aprašymas')]
    df02 = pd.DataFrame(np.array(data2).reshape(int(len(data2)/6),6))
    dfs2.append(df02)
    ############

    



############
#Paros gaisrų aprašymų lentelė
df2 = pd.concat(dfs2, ignore_index=True)
############

In [5]:
df2

,0,1,2,3,4,5
0,53 2018-02-27 06:13,2018-02-27 02:34 02:55 03:46 05:24 12:12,"Ukmergės r., Želvos sen.. Pašilės I vs. Indivi...","Atvykus, gyvenamas namas degė atvira liepsna, ...",AC - 3 3-B,"VPGT-4, SPT-3, SUF-1 Pamainos vadas"
1,92 2018-02-27 07:15,2018-02-27 06:14 06:27 06:33 06:35 06:49,"Alytaus r., kelio Alytus-Lazdijai 16 km. Trans...","Atvykus, iš krovininio automobilio MB 412 vari...",AC - 4 1-B,"VPGT-12, SPT-2 Pamainos vadas"
2,26 2018-02-27 07:35,2018-02-27 06:56 06:59 07:11 07:11 07:32,"Šilalės r., Pajūrio sen., Pajūrio mstl., Jūros...","Atvykus, iš gyvenamo namo rūko dūmai. Namo pir...",AC - 3 1-B 1 (3) KOAA grandis,"VPGT-5, SPT-3, SUF-1 Pamainos vadas, Kitų PGP ..."
3,3 2018-02-27 10:19,2018-02-27 08:58 09:10 09:11 09:59 10:10,"Trakų r., Trakų sen., Raudoniškių k., Dvarčių ...","Atvykus, atvira liepsna degė karkasinis statyb...",AC - 3 2-B 1 (2) KOAA grandis,VPGT-8 Pamainos vadas
4,30 2018-02-27 12:25,2018-02-27 10:56 11:03 11:04 11:12 11:42,"Akmenės r., Naujoji Akmenė, Nepriklausomybės a...","Atvykus, trijų kambarių bute, prie kūrenamo ži...",AC - 3 1-B 1 (2) KOAA grandis,"VPGT-5, SPT-2 Pamainos vadas"
5,57 2018-02-27 14:09,2018-02-27 12:04 12:09 12:16 12:18 12:30,"Jurbarko m., Vydūno g. 56. Ligoninė","Atvykus, nuo ligoninės pastato rūko dūmai. 12:...",AC - 4 1-B 1 (2) KOAA grandis,"VPGT-7, SPT-7 Pamainos vadas"
6,47 2018-02-27 15:21,2018-02-27 12:47 12:58 13:17 13:41 16:16,"Utenos r., Leliūnų sen., Vosgėlių k., Lakšting...","Atvykus, iš mūrinio dviejų aukštų sodo namelio...",AC - 3 2-B 1 (3) KOAA grandis,"VPGT-5, SPT-2 Pamainos vadas"
7,91 2018-02-27 15:46,2018-02-27 13:53 14:07 14:38 15:11 15:24,"Vilniaus r., Avižienių sen., Pikutiškių k., Ce...","Atvykus, medžio apdirbimo bendrovės pastatas b...","AC - 5, ŠA, SLA 2-B 2 (4) KOAA grandys","VPGT-21, SPT-4 APGV komandos viršininko pavadu..."
8,125 2018-02-27 16:33,2018-02-27 13:47 14:01 14:13 14:56 16:41,"Kauno r., Batniavos sen., Tirkšlių k. Garažas,...","Atvykus, garažas degė atvira liepsna. Pastatas...",AC - 5 3-B,"VPGT-13, SPT-3, SUF-1 Skyrininkas"
9,69 2018-02-27 17:34,2018-02-27 16:46 16:48 16:52 16:56 17:15,"Klaipėdos m., P. Cvirkos g. 22. Transporto pr...","Atvykus, lengvasis automobilis Mercedes Benz d...",AC - 2 2-B,"VPGT-6, Gyventojai-1 Pamainos vadas"


In [6]:
df2.to_csv('2018.csv', index=False, header=True, encoding="cp1257")